<a href="https://colab.research.google.com/github/altaga/Open-Driving-Monitor/blob/main/Emotions/test/Test_Emotions_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download model and images for the test

In [ ]:
!wget "https://raw.githubusercontent.com/altaga/Open-Driving-Monitor/main/Emotions/model/emotions-v1.pb"
!wget "https://github.com/altaga/Open-Driving-Monitor/raw/main/Emotions/test/testImages.zip" -O temp.zip
!unzip temp.zip
!rm temp.zip

Archive:  testImages.zip
   creating: testImages/
 extracting: testImages/Angry1.png   
 extracting: testImages/Angry2.png   
 extracting: testImages/Angry3.png   
 extracting: testImages/Angry4.png   
 extracting: testImages/Disgust1.png  
 extracting: testImages/Disgust2.png  
 extracting: testImages/Disgust3.png  
 extracting: testImages/Disgust4.png  
 extracting: testImages/Fear1.png    
 extracting: testImages/Fear2.png    
 extracting: testImages/Fear3.png    
 extracting: testImages/Fear4.png    
 extracting: testImages/Happy1.png   
 extracting: testImages/Happy2.png   
 extracting: testImages/Happy3.png   
 extracting: testImages/Happy4.png   
 extracting: testImages/Neutral1.png  
 extracting: testImages/Neutral2.png  
 extracting: testImages/Neutral3.png  
 extracting: testImages/Neutral4.png  
 extracting: testImages/Sad1.png     
 extracting: testImages/Sad2.png     
 extracting: testImages/Sad3.png     
 extracting: testImages/Sad4.png     
 extracting: testImages/Surpri

Testing all images and display the results.

In [ ]:
import cv2
import numpy as np
import time
import os
import json
from matplotlib import pyplot as plt

precisionByClass = {}
counter = 0

class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
for classes in class_names:
    precisionByClass[classes] = 0

testImages = sorted(os.listdir("testImages"))

# Plot Settings
fig = plt.figure(figsize=(4*4,7*4))


model = cv2.dnn.readNetFromTensorflow('emotions-v1.pb')

# load the image from disk

i = 0
for imagePath in testImages:
    i+=1
    startReal = time.process_time()
    image = cv2.imread("testImages/"+imagePath)
    color = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray3 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    resized = cv2.resize(gray3, (48, 48),  interpolation=cv2.INTER_AREA)
    blob = cv2.dnn.blobFromImage(image=resized, scalefactor=1/256, size=(48, 48))
    model.setInput(blob)
    start = time.process_time()
    outputs = model.forward(model.getUnconnectedOutLayersNames())
    final_outputs = outputs[0][0]
    label_id = np.argmax(final_outputs)
    probs = np.exp(final_outputs) / np.sum(np.exp(final_outputs))
    final_prob = np.max(probs) * 100.
    out_name = class_names[label_id]
    if imagePath.replace(".png","")[:-1]== out_name : counter +=1
    if imagePath.replace(".png","")[:-1]== out_name : precisionByClass[out_name] +=1
    fig.add_subplot(7, 4, i)
    plt.imshow(color)
    plt.title("Predicted: {}\nReal: {}\nProcessing: {:.4f}".format(out_name, imagePath.replace(".png","")[:-1],time.process_time() - start))
    plt.subplots_adjust( hspace=1)

plt.show()

Showing a bar chart with the correct predictions and efficiency of the neural network.

In [ ]:
temp = precisionByClass
for classes in class_names:
    temp[classes] /= 4
    temp[classes] *= 100

emotions = list(temp.keys())
values = list(temp.values())

fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(emotions, values, color ='maroon',
        width = 0.4)

plt.xlabel("Predicted emotions")
plt.ylabel("Correctly predicted emotions %")
plt.title("Network Precision: {:.2f}%".format((counter/len(testImages))*100))
plt.show()